# **LLMs API**


# Get a Gemini API key
1. Go to: https://aistudio.google.com/app/apikey
2. Create a new project.
2. Click on "Create API key"

In [3]:
# Required package installation
# !pip install -q -U google-genai

In [4]:
# Import of required packages
import os
from collections.abc import Iterable

from dotenv import load_dotenv
from google import genai

### Set your API keys here:

- Google API key for Gemini 2.5 Flash access

DO NOT share your keys publicly.


### Get a Gemini API key
1. Go to: https://aistudio.google.com/app/apikey
2. Click on "Create API key"

In [5]:
load_dotenv()

def _get_secret_if_colab(name: str) -> str | None:
    """Return secret from Colab Secrets if available; otherwise None."""
    try:
        from google.colab import userdata
        return userdata.get(name)
    except Exception:
        return None

def get_key(name: str, required: bool = True) -> str | None:
    """
    Get a key by name:
    - Colab Secrets (preferred)
    - Environment variable (fallback)
    """
    val = os.getenv(name)

    if required and not val:
        raise RuntimeError(
            f"Missing required key: {name}\n"
            f"• In Colab: set '{name}' in Secrets\n"
            f"• Elsewhere: set env var {name}"
        )
    return val

def get_keys(names: Iterable[str], required: Iterable[str] = ()) -> dict[str, str | None]:
    """
    Batch-load keys.
    `required` is a subset of names that must exist.
    """
    required = set(required)
    out = {}
    for n in names:
        out[n] = get_key(n, required=(n in required))
    return out



# Example: define the keys you care about
ALL_KEYS = [
    "GOOGLE_API_KEY",
    # "GEMINI_API_KEY",
    # "OPENAI_API_KEY",
    # "ANTHROPIC_API_KEY",
    # "HUGGINGFACEHUB_API_TOKEN",
    # "PINECONE_API_KEY",
]

# Load them (choose which ones are mandatory for *this* notebook)
keys = get_keys(ALL_KEYS, required=["GOOGLE_API_KEY"])

print("✓ Keys loaded:", [k for k, v in keys.items() if v])


✓ Keys loaded: ['GOOGLE_API_KEY']


In [6]:
# Set up a gemini client
client = genai.Client(api_key=keys["GOOGLE_API_KEY"])

In [7]:
# Send a first message
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="I want a travel plan starting in Shanghai ending in Beijin. I have 3 weeks. Answer Shorty!"
)
print(response.text)

Here's a concise 3-week plan from Shanghai to Beijing:

*   **Days 1-4: Shanghai** (The Bund, French Concession, Yu Garden)
*   **Days 5-7: Hangzhou & Suzhou** (West Lake, canals, gardens)
*   **Days 8-10: Huangshan** (Yellow Mountains, ancient villages)
*   **Days 11-14: Xi'an** (Terracotta Army, City Wall, Muslim Quarter)
*   **Days 15-21: Beijing** (Great Wall, Forbidden City, Temple of Heaven)


In [8]:
response

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            text="""Here's a concise 3-week plan from Shanghai to Beijing:

*   **Days 1-4: Shanghai** (The Bund, French Concession, Yu Garden)
*   **Days 5-7: Hangzhou & Suzhou** (West Lake, canals, gardens)
*   **Days 8-10: Huangshan** (Yellow Mountains, ancient villages)
*   **Days 11-14: Xi'an** (Terracotta Army, City Wall, Muslim Quarter)
*   **Days 15-21: Beijing** (Great Wall, Forbidden City, Temple of Heaven)"""
          ),
        ],
        role='model'
      ),
      finish_reason=<FinishReason.STOP: 'STOP'>,
      index=0
    ),
  ],
  model_version='gemini-2.5-flash',
  response_id='M01AafuaLKb5xN8Pz6CC8QE',
  sdk_http_response=HttpResponse(
    headers=<dict len=11>
  ),
  usage_metadata=GenerateContentResponseUsageMetadata(
    candidates_token_count=133,
    prompt_token_count=25,
    prompt_tokens_details=[
      Modali

## LangChain
A package that wraps API calls to different providers and provide robust capabilities!

In [9]:
# !pip install -q -U langchain-google-genai


In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [11]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0, # Set how creative the model is. 0 means deterministic
    max_retries=0,
    api_key=keys["GOOGLE_API_KEY"]
)
llm

ChatGoogleGenerativeAI(profile={'max_input_tokens': 1048576, 'max_output_tokens': 65536, 'image_inputs': True, 'audio_inputs': True, 'pdf_inputs': True, 'video_inputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'image_tool_message': True, 'tool_choice': True}, google_api_key=SecretStr('**********'), model='gemini-2.5-flash', temperature=0.0, max_retries=0, client=<google.genai.client.Client object at 0x1152b1390>, default_metadata=(), model_kwargs={})

### Simple invocation

In [12]:
ai_msg = llm.invoke("I want a travel plan starting in Shanghai ending in Beijin. I have 3 weeks. Answer Shorty!")
ai_msg


AIMessage(content="Okay, Shorty! Here's a 3-week High-Speed Rail (HSR) journey from Shanghai to Beijing:\n\n*   **Days 1-4: Shanghai**\n    *   *Explore:* The Bund, French Concession, Yu Garden, Museums, modern skyscrapers.\n*   **Days 5-6: Suzhou** (HSR from Shanghai ~30 min)\n    *   *Explore:* Classical Gardens (Humble Administrator's Garden), canals, Silk Museum.\n*   **Days 7-9: Hangzhou** (HSR from Suzhou ~2 hrs)\n    *   *Explore:* West Lake (boat ride, cycling), Lingyin Temple, Longjing tea plantations.\n*   **Days 10-12: Huangshan (Yellow Mountain)** (HSR from Hangzhou ~2.5 hrs to Huangshan North)\n    *   *Explore:* Scenic Yellow Mountain peaks, ancient villages (Hongcun or Xidi).\n*   **Days 13-15: Nanjing** (HSR from Huangshan North ~3 hrs)\n    *   *Explore:* Ming Xiaoling Mausoleum, Sun Yat-sen Mausoleum, Presidential Palace, City Wall.\n*   **Days 16-21: Beijing** (HSR from Nanjing ~3.5-4 hrs)\n    *   *Explore:* Forbidden City, Great Wall (Mutianyu or Badaling), Temple 

In [13]:
ai_msg.content

"Okay, Shorty! Here's a 3-week High-Speed Rail (HSR) journey from Shanghai to Beijing:\n\n*   **Days 1-4: Shanghai**\n    *   *Explore:* The Bund, French Concession, Yu Garden, Museums, modern skyscrapers.\n*   **Days 5-6: Suzhou** (HSR from Shanghai ~30 min)\n    *   *Explore:* Classical Gardens (Humble Administrator's Garden), canals, Silk Museum.\n*   **Days 7-9: Hangzhou** (HSR from Suzhou ~2 hrs)\n    *   *Explore:* West Lake (boat ride, cycling), Lingyin Temple, Longjing tea plantations.\n*   **Days 10-12: Huangshan (Yellow Mountain)** (HSR from Hangzhou ~2.5 hrs to Huangshan North)\n    *   *Explore:* Scenic Yellow Mountain peaks, ancient villages (Hongcun or Xidi).\n*   **Days 13-15: Nanjing** (HSR from Huangshan North ~3 hrs)\n    *   *Explore:* Ming Xiaoling Mausoleum, Sun Yat-sen Mausoleum, Presidential Palace, City Wall.\n*   **Days 16-21: Beijing** (HSR from Nanjing ~3.5-4 hrs)\n    *   *Explore:* Forbidden City, Great Wall (Mutianyu or Badaling), Temple of Heaven, Summer 

In [14]:
messages = [
    (
        "system",
        """You are a helpful assistant that helps users plan traveling routes around the world.
        Give general route of the visited cities and some highlights in each.
        Never mention specific businesses , restaurants, hotels, etc.
        Answer shortly please, in a numbered list.""",
    ),
    ("human", "I want a travel plan starting in Shanghai ending in Beijin. I have 3 weeks."),
]
ai_msg = llm.invoke(messages)
ai_msg


AIMessage(content="Here is a possible 3-week travel plan from Shanghai to Beijing:\n\n1.  **Shanghai (approx. 4-5 days):** Explore the Bund waterfront, futuristic skyscrapers, traditional Old City, and vibrant art districts.\n2.  **Hangzhou (approx. 3 days):** Relax by the picturesque West Lake, visit ancient temples, and stroll through tea plantations.\n3.  **Suzhou (approx. 2-3 days):** Discover classical Chinese gardens, tranquil canals, and silk craftsmanship.\n4.  **Nanjing (approx. 2-3 days):** Immerse yourself in history at ancient city walls, mausoleums, and significant historical sites.\n5.  **Xi'an (approx. 4 days):** Fly to this ancient capital to witness the remarkable Terracotta Army, cycle atop the Ming Dynasty City Wall, and explore the Muslim Quarter.\n6.  **Beijing (approx. 5-6 days):** Conclude your journey by exploring the Forbidden City, Tiananmen Square, the Great Wall, the Temple of Heaven, and imperial gardens.", additional_kwargs={}, response_metadata={'finish_r

In [15]:
# Print the actuall message
print(ai_msg.content)

Here is a possible 3-week travel plan from Shanghai to Beijing:

1.  **Shanghai (approx. 4-5 days):** Explore the Bund waterfront, futuristic skyscrapers, traditional Old City, and vibrant art districts.
2.  **Hangzhou (approx. 3 days):** Relax by the picturesque West Lake, visit ancient temples, and stroll through tea plantations.
3.  **Suzhou (approx. 2-3 days):** Discover classical Chinese gardens, tranquil canals, and silk craftsmanship.
4.  **Nanjing (approx. 2-3 days):** Immerse yourself in history at ancient city walls, mausoleums, and significant historical sites.
5.  **Xi'an (approx. 4 days):** Fly to this ancient capital to witness the remarkable Terracotta Army, cycle atop the Ming Dynasty City Wall, and explore the Muslim Quarter.
6.  **Beijing (approx. 5-6 days):** Conclude your journey by exploring the Forbidden City, Tiananmen Square, the Great Wall, the Temple of Heaven, and imperial gardens.


### Chaining
We can chain our model with a prompt template

In [16]:
from langchain_core.prompts import ChatPromptTemplate


In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a helpful assistant that Creates polyndrom from a given text.
            Only answer with the original text and the final result.
            Please also add a sentence compromises of the antonym of each word.
            Answer in a dictionaly format.""",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
ai_msg = chain.invoke(
    {
        # "input_language": "English",
        # "output_language": "Hebrew",
        "input": "The Buddha is great.",
    }
)

ai_msg.content

'Original text: The Buddha is great.\n\n{\n    "Palindrome": "The Buddha is great, taerg si ahdduB ehT.",\n    "Antonym sentence": "A sinner isn\'t awful."\n}'

In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a helpful assistant that translates {input_language} to {output_language}.
        Translate the user sentence. Only answer with final translation.""",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
ai_msg = chain.invoke(
    {
        "input_language": "English",
        "output_language": "Hebrew",
        "input": "The Buddha is great.",
    }
)

ai_msg.content

'הבודהה גדול.'

### Batch call
Calling the same prompt on many inputs

In [19]:
inputs = [
    {
        "input_language": "English",
        "output_language": "Sanskrit",
        "input": "The Buddha is great.",
    },
    {
        "input_language": "English",
        "output_language": "Hebrew",
        "input": "The Buddha is great.",
    },
    {
        "input_language": "English",
        "output_language": "Tibetan",
        "input": "The Buddha is great.",
    },
    {
        "input_language": "English",
        "output_language": "Russian",
        "input": "The Buddha is great.",
    },
]

ai_msgs = chain.batch(inputs)

for msg in ai_msgs:
    print(msg.content)

ChatGoogleGenerativeAIError: Error calling model 'gemini-2.5-flash' (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 5, model: gemini-2.5-flash\nPlease retry in 39.926498243s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPe

## Image Generation (Multimodal Output)

In [ ]:
import base64

from IPython.display import Image, display
from langchain_core.messages import AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI

In [20]:
llm = ChatGoogleGenerativeAI(
    # model="models/gemini-2.0-flash-preview-image-generation",
    model="models/gemini-2.5-flash-image",

    api_key=keys["GOOGLE_API_KEY"]
    )

message = {
    "role": "user",
    "content": "Generate a photorealistic image of a cuddly cat wearing a hat.",
}

response = llm.invoke(
    [message],
    generation_config=dict(response_modalities=["TEXT", "IMAGE"]),
)

ChatGoogleGenerativeAIError: Error calling model 'models/gemini-2.5-flash-image' (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-flash-preview-image\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-flash-preview-image\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-flash-preview-image\nPlease retry in 45.225969857s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'li

In [ ]:
def _get_image_base64(response: AIMessage) -> None:
    image_block = next(
        block
        for block in response.content
        if isinstance(block, dict) and block.get("image_url")
    )
    return image_block["image_url"].get("url").split(",")[-1]

In [ ]:
image_base64 = _get_image_base64(response)
display(Image(data=base64.b64decode(image_base64), width=300))

### Image and text to image

In [ ]:
next_message = {
    "role": "user",
    "content": "Can you take the same image and make the cat black?",
}

response = llm.invoke(
    [message, response, next_message],
    generation_config=dict(response_modalities=["TEXT", "IMAGE"]),
)

image_base64 = _get_image_base64(response)
display(Image(data=base64.b64decode(image_base64), width=300))